# Supervised & Unsupervised Topic Modelling

In [1]:
import arxiv
import string
import unidecode
import numpy as np
import pandas as pd

from typing import List

try:
    from nltk.corpus import stopwords
except:
    import nltk
    nltk.download('stopwords')
finally:
    from nltk.corpus import stopwords

In [2]:
# constants
queries = [
    'automl', 'machinelearning', 'data', 'phyiscs','mathematics', 'recommendation system', 'nlp', 'neural networks'
]

## Load Data

In [3]:
def search_arxiv(queries: List[str], max_results: int = 100) -> pd.DataFrame:
    '''
    This function will search arxiv associated to a set of queries and store
    the latest 10000 (max_results) associated to that search.
    
    params:
        queries (List -> Str) : A list of strings containing keywords you want
                                to search on Arxiv
        max_results (Int) : The maximum number of results you want to see associated
                            to your search. Default value is 1000, capped at 300000
                            
    returns:
        This function will return a DataFrame holding the following columns associated
        to the queries the user has passed. 
            `title`, `date`, `article_id`, `url`, `main_topic`, `all_topics`
    
    example:
        research_df = search_arxiv(
            queries = ['automl', 'recommender system', 'nlp', 'data science'],
            max_results = 10000
        )
    '''
    d = []
    searches = []
    # hitting the API
    for query in queries:
        search = arxiv.Search(
          query = query,
          max_results = max_results,
          sort_by = arxiv.SortCriterion.SubmittedDate,
          sort_order = arxiv.SortOrder.Descending
        )
        searches.append(search)
    
    # Converting search result into df
    for search in searches:
        for res in search.results():
            data = {
                'title' : res.title,
                'date' : res.published,
                'article_id' : res.entry_id,
                'url' : res.pdf_url,
                'main_topic' : res.primary_category,
                'summary' : res.summary,
                'all_topics' : res.categories,
                'authors' : res.authors
            }
            d.append(data)
        
    d = pd.DataFrame(d)
    d['year'] = pd.DatetimeIndex(d['date']).year
    
    # change article id from url to integer
    unique_article_ids = d.article_id.unique()
    article_mapping = {art:idx for idx,art in enumerate(unique_article_ids)}
    d['article_id'] = d['article_id'].map(article_mapping)
    return d


In [4]:
# fetch data from arXiv
research_df = search_arxiv(
    queries = queries,
    max_results = 100
)
print(research_df.shape)

(648, 9)


In [5]:
research_df.head()

,title,date,article_id,url,main_topic,summary,all_topics,authors,year
0,Improvement of Computational Performance of Ev...,2023-01-12 15:59:04+00:00,0,http://arxiv.org/pdf/2301.05102v1,cs.LG,Resource-intensive computations are a major fa...,"[cs.LG, cs.NE, cs.PF]","[Nikolay O. Nikitin, Sergey Teryoshkin, Valeri...",2023
1,Data-driven photometric redshift estimation fr...,2022-12-30 13:01:41+00:00,1,http://arxiv.org/pdf/2212.14668v1,astro-ph.IM,Redshift measurement has always been a constan...,"[astro-ph.IM, astro-ph.CO]","[Felipe M F de Oliveira, Marcelo Vargas dos Sa...",2022
2,Multi-objective Tree-structured Parzen Estimat...,2022-12-13 17:33:02+00:00,2,http://arxiv.org/pdf/2212.06751v1,cs.LG,Hyperparameter optimization (HPO) is essential...,"[cs.LG, cs.AI]","[Shuhei Watanabe, Noow Awad, Masaki Onishi, Fr...",2022
3,POPNASv3: a Pareto-Optimal Neural Architecture...,2022-12-13 17:14:14+00:00,3,http://arxiv.org/pdf/2212.06735v1,cs.LG,The automated machine learning (AutoML) field ...,"[cs.LG, cs.AI, cs.CV, cs.NE]","[Andrea Falanti, Eugenio Lomurno, Danilo Ardag...",2022
4,AutoPINN: When AutoML Meets Physics-Informed N...,2022-12-08 03:44:08+00:00,4,http://arxiv.org/pdf/2212.04058v1,cs.LG,Physics-Informed Neural Networks (PINNs) have ...,"[cs.LG, cs.AI]","[Xinle Wu, Dalin Zhang, Miao Zhang, Chenjuan G...",2022


## Data Cleaning

In [6]:
def remove_stopwords(text: str, sw: List[str] = stopwords) -> str:
    '''
    This function will remove stopwords from the text
    
    Args:
        text: String of data you want to remove stopwords from
        sw: List of strings indicating the list of stopwords
        
    Returns:
        The input string with the stopwords removed.
    '''
    return ' '.join([word for word in text.split() if word not in stopwords.words('english')])
    
def remove_punctuation(text: str, punct: str = string.punctuation) -> str:
    '''
    This function will remove punctuations from the text.
    
    Args:
        text: String of data you want to remove punctuations from
        punct: String of punctuations
    
    Returns:
        The input string with the punctuations removed.
    '''
    cleaned_text = ''.join([char for char in text if char not in punct])
    return cleaned_text
    
def unicode(text: str) -> str:
    '''
    This function will make all the data unicoded. Meaning Â -> A
    
    Args:
        text: String of data you want to unicode
    
    Returns:
        The input string unicoded.
    '''
    return unidecode.unidecode(text)
    
def clean(text: str) -> str:
    '''
    This method will clean the input text through unidecoding and stopword and punctuation 
    removal.
    
    Args:
        text: String indicating the body of text you want to clean
    
    Returns:
        A string corresponding to the cleaned version of the input string.
    '''
    text = unicode(text)
    text = remove_punctuation(text)
    text = remove_stopwords(text)
    return text.lower()

In [7]:
%time research_df['cleaned_summary'] = research_df['summary'].apply(clean)

CPU times: user 3.76 s, sys: 715 ms, total: 4.48 s
Wall time: 4.48 s


In [8]:
research_df.head()

,title,date,article_id,url,main_topic,summary,all_topics,authors,year,cleaned_summary
0,Improvement of Computational Performance of Ev...,2023-01-12 15:59:04+00:00,0,http://arxiv.org/pdf/2301.05102v1,cs.LG,Resource-intensive computations are a major fa...,"[cs.LG, cs.NE, cs.PF]","[Nikolay O. Nikitin, Sergey Teryoshkin, Valeri...",2023,resourceintensive computations major factor li...
1,Data-driven photometric redshift estimation fr...,2022-12-30 13:01:41+00:00,1,http://arxiv.org/pdf/2212.14668v1,astro-ph.IM,Redshift measurement has always been a constan...,"[astro-ph.IM, astro-ph.CO]","[Felipe M F de Oliveira, Marcelo Vargas dos Sa...",2022,redshift measurement always constant need mode...
2,Multi-objective Tree-structured Parzen Estimat...,2022-12-13 17:33:02+00:00,2,http://arxiv.org/pdf/2212.06751v1,cs.LG,Hyperparameter optimization (HPO) is essential...,"[cs.LG, cs.AI]","[Shuhei Watanabe, Noow Awad, Masaki Onishi, Fr...",2022,hyperparameter optimization hpo essential bett...
3,POPNASv3: a Pareto-Optimal Neural Architecture...,2022-12-13 17:14:14+00:00,3,http://arxiv.org/pdf/2212.06735v1,cs.LG,The automated machine learning (AutoML) field ...,"[cs.LG, cs.AI, cs.CV, cs.NE]","[Andrea Falanti, Eugenio Lomurno, Danilo Ardag...",2022,the automated machine learning automl field be...
4,AutoPINN: When AutoML Meets Physics-Informed N...,2022-12-08 03:44:08+00:00,4,http://arxiv.org/pdf/2212.04058v1,cs.LG,Physics-Informed Neural Networks (PINNs) have ...,"[cs.LG, cs.AI]","[Xinle Wu, Dalin Zhang, Miao Zhang, Chenjuan G...",2022,physicsinformed neural networks pinns recently...


# Unsupervised Learning - LDA

In [9]:
import nltk
from gensim import models, corpora

In [10]:
article_summaries = research_df['cleaned_summary'].values
article_summaries = [[word for word in nltk.word_tokenize(article)] for article in article_summaries]

In [11]:
# Create a dictionary of all the words in the dataset
dictionary = corpora.Dictionary(article_summaries)

# Create a corpus
corpus = [dictionary.doc2bow(article) for article in article_summaries]

In [12]:
%%time
# Create an LDA model
lda = models.LdaModel(corpus, num_topics=10, id2word=dictionary, passes=20)

CPU times: user 4.62 s, sys: 39.2 ms, total: 4.66 s
Wall time: 4.67 s


In [13]:
# Get the topic distribution for each document
for i, article in enumerate(article_summaries[0:10]):
    print(f"Article {i+1}: {lda.get_document_topics(corpus[i])}")

# Print the top words for each topic
print(lda.print_topics())

Article 1: [(3, 0.46642175), (4, 0.23049094), (6, 0.29011086)]
Article 2: [(3, 0.9944393)]
Article 3: [(3, 0.42782903), (6, 0.43481714), (9, 0.13027664)]
Article 4: [(3, 0.30521417), (5, 0.18022123), (9, 0.5097325)]
Article 5: [(3, 0.33438456), (6, 0.03671778), (7, 0.4607561), (9, 0.16465013)]
Article 6: [(0, 0.08745317), (3, 0.73005104), (9, 0.17600614)]
Article 7: [(1, 0.7006785), (9, 0.2929171)]
Article 8: [(3, 0.48065946), (7, 0.51125216)]
Article 9: [(2, 0.7006705), (3, 0.05207039), (9, 0.24218127)]
Article 10: [(3, 0.994794)]
[(0, '0.011*"we" + 0.005*"models" + 0.005*"the" + 0.004*"two" + 0.004*"in" + 0.004*"methods" + 0.004*"this" + 0.004*"language" + 0.004*"show" + 0.004*"data"'), (1, '0.008*"time" + 0.005*"data" + 0.005*"the" + 0.005*"model" + 0.005*"in" + 0.004*"this" + 0.004*"work" + 0.003*"proposed" + 0.003*"dynamics" + 0.003*"we"'), (2, '0.014*"we" + 0.007*"models" + 0.006*"language" + 0.006*"in" + 0.005*"model" + 0.005*"system" + 0.005*"learning" + 0.005*"using" + 0.005*"

## Visualization

In [14]:
# Create an empty dataframe
df = pd.DataFrame(columns=["article", "topic", "probability"])

# Loop over the articles and get the top 15 topics for each one
for i, article in enumerate(article_summaries):
    topics = lda.get_document_topics(corpus[i])
    topics = sorted(topics, key=lambda x: x[1], reverse=True)[:15]
    for topic in topics:
        df = df.append({"article": i, "topic": topic[0], "probability": topic[1]}, ignore_index=True)

In [15]:
df

,article,topic,probability
0,0.0,3.0,0.466409
1,0.0,6.0,0.290113
2,0.0,4.0,0.230501
3,1.0,3.0,0.994439
4,2.0,6.0,0.434814
...,...,...,...
902,643.0,9.0,0.992965
903,644.0,3.0,0.992618
904,645.0,3.0,0.994226
905,646.0,8.0,0.993177


In [16]:
# Get the list of top words for each topic
topics = lda.print_topics(num_words=5)

# Create a dictionary that maps the topic numbers to the topic names
topic_names = {}
for topic in topics:
    topic_num = topic[0]
    split_w = [w for w in topic[1].split('+')]
    topic_words = [w.split('*')[1] for w in split_w]
    topic_name = ",".join(topic_words)
    topic_names[topic_num] = topic_name

# Add a new column to the dataframe with the topic names
df["topic_name"] = df["topic"].map(topic_names)


In [17]:
df['topic'].value_counts()

3.0    169
9.0    126
2.0    115
8.0     91
4.0     86
7.0     75
0.0     74
5.0     71
6.0     60
1.0     40
Name: topic, dtype: int64

## Word Cloud

## Topic Analysis

# Supervised Learning - Binary Classification

In [ ]:
# generate labels

## Evaluation

# Supervised Learning - Multi-Class Classification

---